In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import os 
import sys
import time

project_dir_path = '/home/onoue/ws/lukasiewicz_1'
sys.path.append(project_dir_path)

for path in sys.path:
    print(path)

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC




from src.setup_problem import Setup
from src.misc import visualize_result

import cvxpy as cp

from src.misc import *

/home/onoue/ws/lukasiewicz_1/notebooks/pima_indian_diabetes
/usr/lib/python310.zip
/usr/lib/python3.10
/usr/lib/python3.10/lib-dynload

/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages
/home/onoue/ws/lukasiewicz_1
/home/onoue/ws/lukasiewicz_1
/home/onoue/ws/lukasiewicz_1


# データの準備

In [66]:
# train_index = pd.read_csv("/home/onoue/ws/lukasiewicz_1/inputs/pima_indian_diabetes/train_index.csv", index_col=0)["0"].to_list()
# # display(train_index)

# test_index = pd.read_csv("/home/onoue/ws/lukasiewicz_1/inputs/pima_indian_diabetes/test_index.csv", index_col=0)["0"].to_list()
# # display(test_index)

In [40]:
data = pd.read_csv("./data/diabetes_discretized.csv", index_col=0)

feature_names = data.columns.to_list()[1:]

X = data.drop(["Outcome"], axis=1)
y = data["Outcome"]

random_state = 45
test_size = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=test_size,
                                                    random_state=random_state)

train_index = X_train.index
test_index = X_test.index

import os
project_dir_path = "/home/onoue/ws/lukasiewicz_1/"
data_dir_path = "inputs/pima_indian_diabetes"
save_dir_path = os.path.join(project_dir_path, data_dir_path)

file_path_1 = os.path.join(save_dir_path, 'train_index_3.csv')
pd.DataFrame(train_index).to_csv(file_path_1)

file_path_2 = os.path.join(save_dir_path, 'test_index_3.csv')
pd.DataFrame(test_index).to_csv(file_path_2)


X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

# SVM（線形カーネル）のモデルを作成
svm_linear = SVC(kernel='linear')

# モデルを訓練
svm_linear.fit(X_train, y_train)

# テストデータで予測
y_pred_linear = svm_linear.predict(X_test)

# Accuracyを計算
accuracy_linear = accuracy_score(y_test, y_pred_linear)

# Confusion Matrixを計算
cm_linear = confusion_matrix(y_test, y_pred_linear)

# Classification Reportを出力
report_linear = classification_report(y_test, y_pred_linear)

# 結果を出力
print("SVM (Linear Kernel) Accuracy:", accuracy_linear)
print()
print()
print("SVM (Linear Kernel) Confusion Matrix:\n", cm_linear)
print()
print()
print("SVM (Linear Kernel) Classification Report:\n", report_linear)

SVM (Linear Kernel) Accuracy: 0.7835820895522388


SVM (Linear Kernel) Confusion Matrix:
 [[85  9]
 [20 20]]


SVM (Linear Kernel) Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.90      0.85        94
           1       0.69      0.50      0.58        40

    accuracy                           0.78       134
   macro avg       0.75      0.70      0.72       134
weighted avg       0.77      0.78      0.77       134



In [41]:
test_index

Index([227,   5, 646, 381, 268, 460, 434, 656, 150, 519,
       ...
       133, 366, 331,  73, 138, 408, 595, 190, 421,  77],
      dtype='int64', length=134)

In [32]:
test_index

Index([419, 186, 556, 738, 320, 420, 364, 232, 605, 109,
       ...
       132, 323,  99,  63, 714, 626, 487, 638, 248,  24],
      dtype='int64', length=134)

In [27]:
test_index

Index([639, 685, 510, 154, 318, 740,  47, 520, 235,  57,
       ...
        17,  13, 411, 730, 464, 389, 495, 143, 721, 360],
      dtype='int64', length=134)

In [42]:
data = pd.read_csv('./data/diabetes_discretized.csv', index_col=0).loc[train_index, :]

outcome = data['Outcome']
features = data.drop(['Outcome'], axis=1)
feature_names = list(features.columns)


data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes/train_3')

if not os.path.exists(data_dir_path):
    os.mkdir(data_dir_path)


df = features.copy()
df['target'] = outcome.replace(0, -1)
display(df)

file_name = "L_" + "Outcome" + '.csv'
file_path = os.path.join(data_dir_path, file_name)
df.to_csv(file_path)


for feature_name in feature_names:
    df = features.copy()
    df['target'] = df[feature_name].replace(0, -1)
    # display(df)

    file_name = "L_" + feature_name + '.csv'
    file_path = os.path.join(data_dir_path, file_name)
    df.to_csv(file_path)


data_num = 15
data_dim = 21

arr_U = np.random.randint(2, size=(data_num, data_dim))
df_U = pd.DataFrame(arr_U)
df_U.to_csv(os.path.join(data_dir_path, 'U.csv'))

,Pregnancies_Low,Pregnancies_Medium,Pregnancies_High,Glucose_Low,Glucose_Medium,Glucose_High,BloodPressure_Low,BloodPressure_Medium,BloodPressure_High,SkinThickness_Low,...,BMI_Low,BMI_Medium,BMI_High,DiabetesPedigreeFunction_Low,DiabetesPedigreeFunction_Medium,DiabetesPedigreeFunction_High,Age_Low,Age_Medium,Age_High,target
47,1,0,0,1,0,0,0,1,0,0,...,1,0,0,0,1,0,1,0,0,-1
423,1,0,0,0,1,0,0,1,0,0,...,0,1,0,1,0,0,1,0,0,-1
24,0,0,1,0,1,0,0,0,1,0,...,0,1,0,1,0,0,0,1,0,1
739,1,0,0,0,1,0,0,1,0,1,...,0,0,1,1,0,0,0,1,0,1
85,1,0,0,0,1,0,0,1,0,0,...,0,1,0,0,1,0,1,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,0,1,0,0,0,1,0,1,0,1,...,0,1,0,1,0,0,1,0,0,1
704,1,0,0,0,1,0,0,1,0,1,...,1,0,0,1,0,0,1,0,0,-1
631,1,0,0,0,1,0,0,1,0,0,...,0,1,0,1,0,0,1,0,0,-1
741,1,0,0,0,1,0,1,0,0,1,...,0,1,0,1,0,0,1,0,0,-1


In [43]:
data = pd.read_csv('./data/diabetes_discretized.csv', index_col=0).loc[test_index, :]

outcome = data['Outcome']
features = data.drop(['Outcome'], axis=1)
feature_names = list(features.columns)


data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes/test_3')

if not os.path.exists(data_dir_path):
    os.mkdir(data_dir_path)


df = features.copy()
df['target'] = outcome.replace(0, -1)
display(df)

file_name = "L_" + "Outcome" + '.csv'
file_path = os.path.join(data_dir_path, file_name)
df.to_csv(file_path)


# for feature_name in feature_names:
#     df = features.copy()
#     df['target'] = df[feature_name].replace(0, -1)
#     # display(df)

#     file_name = "L_" + feature_name + '.csv'
#     file_path = os.path.join(data_dir_path, file_name)
#     df.to_csv(file_path)

,Pregnancies_Low,Pregnancies_Medium,Pregnancies_High,Glucose_Low,Glucose_Medium,Glucose_High,BloodPressure_Low,BloodPressure_Medium,BloodPressure_High,SkinThickness_Low,...,BMI_Low,BMI_Medium,BMI_High,DiabetesPedigreeFunction_Low,DiabetesPedigreeFunction_Medium,DiabetesPedigreeFunction_High,Age_Low,Age_Medium,Age_High,target
227,1,0,0,0,0,1,1,0,0,0,...,0,1,0,0,1,0,1,0,0,1
5,0,1,0,0,1,0,0,1,0,1,...,1,0,0,1,0,0,1,0,0,-1
646,1,0,0,0,0,1,0,1,0,1,...,1,0,0,1,0,0,1,0,0,1
381,1,0,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0,1,0,0,-1
268,1,0,0,0,1,0,1,0,0,1,...,1,0,0,1,0,0,1,0,0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,0,1,0,0,0,1,0,1,0,1,...,1,0,0,0,0,1,0,1,0,1
595,1,0,0,0,0,1,0,1,0,1,...,0,1,0,0,1,0,1,0,0,1
190,1,0,0,0,1,0,0,1,0,1,...,1,0,0,1,0,0,1,0,0,-1
421,1,0,0,1,0,0,0,1,0,1,...,1,0,0,0,1,0,1,0,0,-1


In [44]:
data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes')

train_data_dir_path = os.path.join(data_dir_path, "train_3")

file_list = os.listdir(train_data_dir_path)

L_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('L') and filename.endswith('.csv')]

U_files = [filename.split('.csv')[0] for filename in file_list 
           if filename.startswith('U') and filename.endswith('.csv')]

file_names_dict = {
    'supervised': L_files,
    'unsupervised': U_files,
    'rule': ['rules']
}

problem_instance = Setup(train_data_dir_path, file_names_dict)
objective, constraints = problem_instance.main(c1=10, c2=10)

start_time = time.time()
problem = cp.Problem(objective, constraints)
result = problem.solve(verbose=True)
end_time = time.time()
print()
print(f'学習時間: {end_time - start_time} 秒')

Loading data ...
Done in 0.1252119541168213 seconds! 

Loading rules ...
Done in 0.0006964206695556641 seconds! 

Identifying predicates ...
Done in 0.002471446990966797 seconds! 

Constructing objective function ...
Done in 8.953991174697876 seconds! 

Constructing constraints ...
Done in 16.670061826705933 seconds! 

All done


/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages/cvxpy/problems/problem.py:157: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


                                     CVXPY                                     
                                     v1.3.2                                    
(CVXPY) Nov 25 03:39:03 PM: Your problem has 12322 variables, 37416 constraints, and 0 parameters.
(CVXPY) Nov 25 03:39:09 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Nov 25 03:39:09 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Nov 25 03:39:09 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Nov 25 03:39:15 PM: Compiling problem (target solver=ECOS).
(CVXPY) Nov 25 03:39:15 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuf

In [45]:
test_data_dir_path = os.path.join(data_dir_path, "test_3")
file_path = os.path.join(test_data_dir_path, "L_Outcome.csv")
test_df = pd.read_csv(file_path, index_col=0)
display(test_df.head())

test_data = {
    'Outcome': np.array(test_df)
}

test_data

,Pregnancies_Low,Pregnancies_Medium,Pregnancies_High,Glucose_Low,Glucose_Medium,Glucose_High,BloodPressure_Low,BloodPressure_Medium,BloodPressure_High,SkinThickness_Low,...,BMI_Low,BMI_Medium,BMI_High,DiabetesPedigreeFunction_Low,DiabetesPedigreeFunction_Medium,DiabetesPedigreeFunction_High,Age_Low,Age_Medium,Age_High,target
227,1,0,0,0,0,1,1,0,0,0,...,0,1,0,0,1,0,1,0,0,1
5,0,1,0,0,1,0,0,1,0,1,...,1,0,0,1,0,0,1,0,0,-1
646,1,0,0,0,0,1,0,1,0,1,...,1,0,0,1,0,0,1,0,0,1
381,1,0,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0,1,0,0,-1
268,1,0,0,0,1,0,1,0,0,1,...,1,0,0,1,0,0,1,0,0,-1


{'Outcome': array([[ 1,  0,  0, ...,  0,  0,  1],
        [ 0,  1,  0, ...,  0,  0, -1],
        [ 1,  0,  0, ...,  0,  0,  1],
        ...,
        [ 1,  0,  0, ...,  0,  0, -1],
        [ 1,  0,  0, ...,  0,  0, -1],
        [ 0,  1,  0, ...,  0,  0, -1]])}

In [46]:
p_dict = problem_instance.predicates_dict
selected_predicates = ['Outcome']
selected_p_dict = {key: value for key, value in p_dict.items() if key in selected_predicates}
selected_p_dict

{'Outcome': <src.misc.Predicate at 0x7f0d8835fdc0>}

In [38]:
def test_trained_predicate(predicates_dict, test_data_dict):
    result_dict = {}
    p_names = predicates_dict.keys()

    for p_name in p_names:
        pred_vals = []
        preds = []

        p = predicates_dict[p_name]
        test_data = test_data_dict[p_name]

        if isinstance(test_data, pd.DataFrame):
            test_data = np.array(test_data)

        cnt = 0

        for data in test_data:

            x, ans = data[:-1], data[-1]
            pred_val = p(x).value
            pred_vals.append(pred_val)

            if (pred_val >= 0.5 and ans == 1) or (pred_val < 0.5 and ans == -1):
                cnt += 1

            pred = (pred_val >= 0.5 and ans == 1) or (pred_val < 0.5 and ans == -1)
            preds.append(pred)

        p_arr = np.hstack([test_data, 
                           np.array(pred_vals).reshape(-1,1), 
                           np.array(preds).reshape(-1, 1)])
        
        # p_df = pd.DataFrame(p_arr, columns=['r', 'g', 'b', 'Ans', 'pred_val', 'pred'])

        result_dict[p_name] = p_arr

        print(cnt)
        print(f'Accuracy of {p_name}: {cnt / len(test_data)}')

    return result_dict

In [47]:
res_dict = test_trained_predicate(selected_p_dict, test_data)

105
Accuracy of Outcome: 0.7835820895522388


In [ ]:
0.7835820895522388

In [7]:
problem_instance.predicates_dict

{'Glucose_Low': <src.misc.Predicate at 0x7f9dc01915a0>,
 'Age_Low': <src.misc.Predicate at 0x7f9dc01914e0>,
 'BMI_Low': <src.misc.Predicate at 0x7f9dc0190550>,
 'Outcome': <src.misc.Predicate at 0x7f9dc0190a30>,
 'Glucose_High': <src.misc.Predicate at 0x7f9dc0191840>,
 'DiabetesPedigreeFunction_Medium': <src.misc.Predicate at 0x7f9dc01904f0>,
 'Pregnancies_High': <src.misc.Predicate at 0x7f9dc01919c0>,
 'DiabetesPedigreeFunction_High': <src.misc.Predicate at 0x7f9dc0191a80>,
 'Age_Medium': <src.misc.Predicate at 0x7f9dc01910c0>,
 'SkinThickness_Low': <src.misc.Predicate at 0x7f9dc0192e30>,
 'BloodPressure_Low': <src.misc.Predicate at 0x7f9dc01900a0>,
 'Pregnancies_Medium': <src.misc.Predicate at 0x7f9dc0190160>,
 'BloodPressure_Medium': <src.misc.Predicate at 0x7f9dc0190220>,
 'DiabetesPedigreeFunction_Low': <src.misc.Predicate at 0x7f9dc01902e0>,
 'BMI_High': <src.misc.Predicate at 0x7f9dc01903a0>,
 'BMI_Medium': <src.misc.Predicate at 0x7f9dc0192170>,
 'SkinThickness_Medium': <src.mi

In [19]:
problem_instance.w_j

Variable((22, 22))

In [18]:
X_test.shape

(134, 21)

In [13]:
X_test = test_data['Outcome'][:, :-1]
y_test = test_data['Outcome'][:, -1]

In [24]:
y_pred = p_dict['Outcome'](X_test.T).value
y_pred

array([ 2.04672390e-12,  1.00000000e+00,  1.32087674e-11,  2.21579421e-11,
        1.09958986e-11,  2.29462560e-11,  1.00000000e+00,  1.54282143e-12,
        2.31245578e-11,  3.59712260e-12, -3.40333317e-12,  2.43663423e-11,
        2.55265253e-11,  2.04672390e-12,  4.83280915e-11,  2.71705436e-11,
        4.55528948e-11,  3.78745368e-11,  3.66498498e-12, -4.36997660e-12,
        3.91071620e-11,  1.39539769e-11,  5.76177994e-13,  1.00000000e+00,
        4.62219152e-12,  1.27048372e-11,  2.24422148e-11,  1.00000000e+00,
        5.76177994e-13,  4.11795598e-12,  1.00000000e+00,  2.14755991e-11,
        1.00000000e+00,  1.00000000e+00,  3.78116705e-11,  1.66879843e-11,
        1.08010823e-12,  3.68450548e-11,  1.00000000e+00,  4.55528948e-11,
        1.00000000e+00,  3.97861466e-11,  5.84652882e-11,  7.01350089e-12,
        1.12173049e-11,  1.00000000e+00,  1.00000000e+00,  4.11795598e-12,
        2.75977574e-11,  1.87547200e-11,  1.32042710e-11,  1.00000000e+00,
        1.02506892e-11,  

In [31]:
y_pred_interpreted = np.where(y_pred >= 0.5, 1, -1)
y_pred_interpreted

array([-1,  1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1, -1, -1,  1, -1,  1,  1,
       -1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1,  1,  1, -1, -1, -1, -1,
        1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1,  1,  1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1,  1, -1, -1,  1, -1,  1, -1,  1, -1, -1,  1,
       -1, -1,  1, -1,  1, -1,  1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1, -1, -1, -1, -1, -1,
       -1, -1,  1, -1, -1,  1,  1, -1, -1, -1, -1,  1, -1, -1, -1])

In [32]:
y_test

array([ 1,  1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1,  1,
       -1, -1, -1,  1, -1, -1,  1,  1, -1, -1,  1, -1,  1,  1, -1, -1, -1,
       -1, -1, -1,  1,  1,  1, -1,  1,  1, -1, -1,  1,  1, -1, -1, -1, -1,
        1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1, -1, -1, -1, -1,
        1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1, -1, -1,  1,
        1, -1,  1,  1,  1, -1,  1, -1, -1, -1, -1,  1,  1, -1, -1, -1,  1,
        1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1, -1, -1, -1, -1, -1,
       -1, -1,  1,  1, -1,  1,  1,  1, -1, -1, -1, -1,  1, -1,  1])

In [40]:
accuracy = accuracy_score(y_test, y_pred_interpreted)
print("Accuracy: ")
print(accuracy)
print()

conf_matrix = confusion_matrix(y_test, y_pred_interpreted)
print("Confusion matrix: ")
print(conf_matrix)
print()

class_report = classification_report(y_test, y_pred_interpreted)
print("classification report: ")
print(class_report)


Accuracy: 
0.7761194029850746

Confusion matrix: 
[[82  8]
 [22 22]]

classification report: 
              precision    recall  f1-score   support

          -1       0.79      0.91      0.85        90
           1       0.73      0.50      0.59        44

    accuracy                           0.78       134
   macro avg       0.76      0.71      0.72       134
weighted avg       0.77      0.78      0.76       134



In [38]:
confusion_matrix(y_test, y_pred_interpreted)

array([[82,  8],
       [22, 22]])

In [39]:
classification_report(y_test, y_pred_interpreted)

'              precision    recall  f1-score   support\n\n          -1       0.79      0.91      0.85        90\n           1       0.73      0.50      0.59        44\n\n    accuracy                           0.78       134\n   macro avg       0.76      0.71      0.72       134\nweighted avg       0.77      0.78      0.76       134\n'

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2.046752e-12,0.0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.000000e+00,1.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,-1.0,1.320880e-11,1.0
3,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,-1.0,2.215794e-11,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,-1.0,1.099587e-11,1.0
5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,-1.0,2.294626e-11,1.0
6,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,-1.0,1.000000e+00,0.0
7,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,-1.0,1.542821e-12,1.0
8,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,-1.0,2.312456e-11,1.0
9,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,3.597095e-12,0.0


In [8]:
res_dict = test_trained_predicate(selected_p_dict, test_data)

104
Accuracy of Outcome: 0.7761194029850746


In [9]:
pd.set_option('display.max_rows', 200)


In [111]:
pd.DataFrame(test_data['Outcome'])

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,1,0,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0,1,0,0,1
1,0,1,0,0,0,1,0,1,0,0,...,0,1,0,0,1,0,0,0,1,1
2,1,0,0,0,1,0,0,1,0,0,...,0,1,0,1,0,0,1,0,0,-1
3,1,0,0,0,1,0,1,0,0,1,...,0,1,0,0,1,0,1,0,0,-1
4,1,0,0,0,1,0,1,0,0,1,...,1,0,0,0,1,0,1,0,0,-1
5,1,0,0,0,1,0,0,0,1,0,...,0,0,1,0,1,0,1,0,0,-1
6,1,0,0,0,0,1,0,1,0,0,...,0,1,0,1,0,0,1,0,0,-1
7,1,0,0,1,0,0,0,1,0,0,...,1,0,0,0,1,0,1,0,0,-1
8,1,0,0,0,1,0,1,0,0,0,...,0,1,0,0,1,0,1,0,0,-1
9,1,0,0,1,0,0,0,0,1,0,...,0,1,0,1,0,0,1,0,0,1


In [109]:
test_data['Outcome'][:, -2]

array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0])

In [ ]:
tmp2 = test_data['Outcome'][:, ]

In [106]:
tmp = test_data['Outcome'][:, -1]
tmp

array([ 1,  1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1,  1,
       -1, -1, -1,  1, -1, -1,  1,  1, -1, -1,  1, -1,  1,  1, -1, -1, -1,
       -1, -1, -1,  1,  1,  1, -1,  1,  1, -1, -1,  1,  1, -1, -1, -1, -1,
        1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1, -1, -1, -1, -1,
        1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1,  1,  1, -1, -1,  1,
        1, -1,  1,  1,  1, -1,  1, -1, -1, -1, -1,  1,  1, -1, -1, -1,  1,
        1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1,  1, -1, -1, -1, -1, -1,
       -1, -1,  1,  1, -1,  1,  1,  1, -1, -1, -1, -1,  1, -1,  1])

In [107]:
np.count_nonzero(tmp < 0), np.count_nonzero(tmp > 0)

(90, 44)

In [108]:
44 / 90

0.4888888888888889

In [113]:
pd.DataFrame(res_dict['Outcome'])

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2.046752e-12,0.0
1,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.000000e+00,1.0
2,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,-1.0,1.320880e-11,1.0
3,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,-1.0,2.215794e-11,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,-1.0,1.099587e-11,1.0
5,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,-1.0,2.294626e-11,1.0
6,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,-1.0,1.000000e+00,0.0
7,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,-1.0,1.542821e-12,1.0
8,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,-1.0,2.312456e-11,1.0
9,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,3.597095e-12,0.0
